In [41]:
import pandas as pd

In [42]:
path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/AVANCE DE COSECHA - PRUEBA - Copy.xlsx'

In [43]:
xlsx_resumen = pd.read_excel(path_xlsx_avance, sheet_name='AVANCE_FECHAS')

In [44]:
# elimina el ultimo registro, para descartar los totales
xlsx_resumen = xlsx_resumen.drop(xlsx_resumen.index[-1])

In [45]:
xlsx_resumen

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
0,1,2024-05-16,4.127295,221.918352,NaN
1,1,2024-05-29,NaN,NaN,1906.18
2,1,2024-05-31,45.972261,2338.489409,970.59
3,1,2024-06-05,63.240335,2836.576690,2483.70
4,1,2024-06-10,48.153913,2486.759690,2421.99
...,...,...,...,...,...
3250,578,2024-07-17,7.535220,NaN,285.20
3251,579,2024-08-11,NaN,NaN,43.41
3252,580,2024-07-20,NaN,NaN,69.59
3253,580,2024-07-25,NaN,NaN,31.67


In [46]:
def colapsar_tabla(df_param):
    suma_acumulador = [None] * len(df_param)
    acumulador = 0
    contador = 0
    for i, r in df_param.iterrows():
        if(r['AREA'] == 0):
            acumulador += r['ENTREGADO']
            contador += 1
        else:
            acumulador += r['ENTREGADO']
            suma_acumulador[contador] = acumulador
            acumulador = 0
            contador += 1
            continue
    if suma_acumulador[-1] == None:
        suma_acumulador[-1] = acumulador
        
    #print(df_param)
    #print(suma_acumulador)
    df_param['ENTREGAS'] = suma_acumulador
    
    print(df_param)
    df_param = df_param.dropna(subset=['ENTREGAS'])
    df_param = df_param.drop(columns=['ENTREGADO'])
    #df_param = df_param[df_param['AREA']!=0]
    return df_param

In [28]:
suma_acumulador = [None] * len(grupo)

In [29]:
type(suma_acumulador[0])

NoneType

In [30]:
list_cod_cos = list(set(xlsx_resumen['COD_COS']))

In [31]:
list_df = []

In [37]:
grupo = xlsx_resumen[xlsx_resumen['COD_COS']==17].fillna(0)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_30284\3309769114.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==17].fillna(0)


In [38]:
grupo

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
112,17,2024-06-18,22.298621,1288.302440,2358.23
113,17,2024-06-20,12.108021,908.951624,762.37
114,17,2024-07-17,21.077437,1619.120484,2054.73
115,17,2024-07-25,0.000000,0.000000,81.57
116,17,2024-08-11,0.000000,0.000000,1252.70


In [39]:
result = colapsar_tabla(grupo)

     COD_COS      FECHA       AREA     ESTIMADO  ENTREGADO  ENTREGAS
112       17 2024-06-18  22.298621  1288.302440    2358.23   2358.23
113       17 2024-06-20  12.108021   908.951624     762.37    762.37
114       17 2024-07-17  21.077437  1619.120484    2054.73   2054.73
115       17 2024-07-25   0.000000     0.000000      81.57       NaN
116       17 2024-08-11   0.000000     0.000000    1252.70   1334.27


In [40]:
result

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
112,17,2024-06-18,22.298621,1288.302440,2358.23
113,17,2024-06-20,12.108021,908.951624,762.37
114,17,2024-07-17,21.077437,1619.120484,2054.73
116,17,2024-08-11,0.000000,0.000000,1334.27


In [47]:
for i in list_cod_cos:
    grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
    result = colapsar_tabla(grupo)
    list_df.append(result)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_30284\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_30284\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_30284\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

   COD_COS      FECHA        AREA     ESTIMADO  ENTREGADO  ENTREGAS
0        1 2024-05-16    4.127295   221.918352       0.00      0.00
1        1 2024-05-29    0.000000     0.000000    1906.18       NaN
2        1 2024-05-31   45.972261  2338.489409     970.59   2876.77
3        1 2024-06-05   63.240335  2836.576690    2483.70   2483.70
4        1 2024-06-10   48.153913  2486.759690    2421.99   2421.99
5        1 2024-06-18   20.112090   815.611812    2230.23   2230.23
6        1 2024-06-20   29.470226  2021.733284       0.00      0.00
7        1 2024-07-20   10.552630   619.266573    1136.33   1136.33
8        1 2024-07-25   33.115243  1651.084930    1916.31   1916.31
9        1 2024-08-11  111.015957  7411.451546    6811.41   6811.41
    COD_COS      FECHA       AREA     ESTIMADO  ENTREGADO  ENTREGAS
10        2 2024-05-16   1.918209   117.139496       0.00      0.00
11        2 2024-05-31   2.200423   136.933849       0.00      0.00
12        2 2024-06-18  16.654161   709.412270  

C:\Users\Usuario\AppData\Local\Temp\ipykernel_30284\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_30284\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_30284\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

     COD_COS      FECHA      AREA    ESTIMADO  ENTREGADO  ENTREGAS
502       84 2024-05-21  0.837556   45.396813       0.00      0.00
503       84 2024-05-29  0.000000    0.000000      81.88       NaN
504       84 2024-05-31  0.000000    0.000000      57.68       NaN
505       84 2024-06-05  1.525151   71.144539     180.39    319.95
506       84 2024-06-10  1.077834   49.001978     157.02    157.02
507       84 2024-06-18  4.944335  240.643973     152.65    152.65
508       84 2024-06-20  0.000000    0.000000      35.96       NaN
509       84 2024-07-17  6.523210  411.600261     477.94    513.90
510       84 2024-07-20  0.000000    0.000000      63.59       NaN
511       84 2024-08-11  4.890224  333.680575     248.02    311.61
     COD_COS      FECHA      AREA    ESTIMADO  ENTREGADO  ENTREGAS
512       85 2024-06-10  0.000000    0.000000      51.00       NaN
513       85 2024-06-18  0.000000    0.000000     135.35       NaN
514       85 2024-07-17  9.703973  509.999933     558.95    74

C:\Users\Usuario\AppData\Local\Temp\ipykernel_30284\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_30284\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_30284\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

In [48]:
df_combinado = pd.concat(list_df, ignore_index=True)

In [49]:
df_combinado

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
0,1,2024-05-16,4.127295,221.918352,0.00
1,1,2024-05-31,45.972261,2338.489409,2876.77
2,1,2024-06-05,63.240335,2836.576690,2483.70
3,1,2024-06-10,48.153913,2486.759690,2421.99
4,1,2024-06-18,20.112090,815.611812,2230.23
...,...,...,...,...,...
2311,577,2024-07-20,1.435498,0.000000,89.57
2312,578,2024-07-17,7.535220,0.000000,285.20
2313,579,2024-08-11,0.000000,0.000000,43.41
2314,580,2024-07-25,0.000000,0.000000,101.26


In [51]:
df_combinado.to_excel('G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')